In [9]:
import keras.losses
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import img_to_array
from tensorflow.keras import layers
from keras.regularizers import l2
import numpy as np
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
from sklearn.utils import class_weight
import datetime
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
model = VGG16(include_top=True)

In [3]:
def get_features(ID):
    img_path = 'food_porn/food/' + ID + '.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = np.array(model.predict(x))
    return features

In [4]:
%%time
data = np.zeros((10000,1001))
id = np.array(range(0,10000))
data[:,0] = id.transpose()

def features():
    for i in range(0,10000):
        featureid = '{0:05}'.format(i)
        features = get_features(featureid)
        data[i,1:] = features
        if i % 100 == True:
            print(i)
    return data

data = features()

1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901
7001
7101
7201
7301
7401
7501
7601
7701
7801
7901
8001
8101
8201
8301
8401
8501
8601
8701
8801
8901
9001
9101
9201
9301
9401
9501
9601
9701
9801
9901


In [2]:
save_ass = 'features_vgg16.csv'

In [3]:
print(np.count_nonzero(data))
save_ass = 'features_vgg16.csv'

np.savetxt(save_ass, data, delimiter=',')

NameError: name 'data' is not defined

using false we got:     527 correct
using true we got:      520 correct

how about... we use the true version to feed a NN

In [4]:
train_trip = np.loadtxt("train_triplets.txt").astype('int')

data = np.genfromtxt(save_ass, delimiter= ',')
data = np.array(data[:,1:])

def load_features(int_id):
    return data[int_id]

In [5]:
%%time

#create the base dataset with the 2 sets of labels
data_y1 = np.ones((59515,1))
data_y0 = np.zeros((59515,1))

#create the inverse dataset:
train_trip_inv = np.zeros((59515,3)).astype('int')
train_trip_inv[:,0] = train_trip[:,0]
train_trip_inv[:,1] = train_trip[:,2]
train_trip_inv[:,2] = train_trip[:,1]

#assemble everything into one huge pile of S*** in order to shuffle it
id_fin = np.concatenate((train_trip,train_trip_inv),axis=0)
labels = np.concatenate((data_y1,data_y0),axis=0)
id_labels = np.concatenate((id_fin,labels),axis=1)

data_nn_x = np.zeros((119030,3000))

for i in range(0,3):
        for j in range(0,119030):
            data_nn_x[j,range(i*1000,1000*(i+1))] = load_features(id_labels[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)


#print(train_trip)
#print(train_trip_inv)
#now we try to randomize the data in order to increase our accuracy
# we may even double the dataset by using 0 as well as the 1 case

we're at entry: 0 0
we're at entry: 0 10000
we're at entry: 0 20000
we're at entry: 0 30000
we're at entry: 0 40000
we're at entry: 0 50000
we're at entry: 0 60000
we're at entry: 0 70000
we're at entry: 0 80000
we're at entry: 0 90000
we're at entry: 0 100000
we're at entry: 0 110000
we're at entry: 1 0
we're at entry: 1 10000
we're at entry: 1 20000
we're at entry: 1 30000
we're at entry: 1 40000
we're at entry: 1 50000
we're at entry: 1 60000
we're at entry: 1 70000
we're at entry: 1 80000
we're at entry: 1 90000
we're at entry: 1 100000
we're at entry: 1 110000
we're at entry: 2 0
we're at entry: 2 10000
we're at entry: 2 20000
we're at entry: 2 30000
we're at entry: 2 40000
we're at entry: 2 50000
we're at entry: 2 60000
we're at entry: 2 70000
we're at entry: 2 80000
we're at entry: 2 90000
we're at entry: 2 100000
we're at entry: 2 110000
Wall time: 42.5 s


In [7]:
collab = np.concatenate((data_nn_x,labels), axis=1)
np.random.shuffle(collab)

data_x = collab[:,:3000]
data_y = collab[:,3000]

In [10]:
# split data into train and test sets
seed = 7
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=test_size, random_state=seed)

In [12]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

[12:58:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 79.73%


In [14]:
test_trip = np.loadtxt("test_triplets.txt").astype('int')
test_nn_x = np.zeros((59544,3000))
for i in range(0,3):
        for j in range(0,59544):
            test_nn_x[j,range(i*1000,1000*(i+1))] = load_features(test_trip[j,i].astype('int'))
            if j%10000==False:
                print("we're at entry:", i, j)

predict_test_nn = model.predict(test_nn_x)

we're at entry: 0 0
we're at entry: 0 10000
we're at entry: 0 20000
we're at entry: 0 30000
we're at entry: 0 40000
we're at entry: 0 50000
we're at entry: 1 0
we're at entry: 1 10000
we're at entry: 1 20000
we're at entry: 1 30000
we're at entry: 1 40000
we're at entry: 1 50000
we're at entry: 2 0
we're at entry: 2 10000
we're at entry: 2 20000
we're at entry: 2 30000
we're at entry: 2 40000
we're at entry: 2 50000


In [16]:
export = np.array(predict_test_nn>0.5).astype('int')
np.savetxt("submission_vgg16_xgboost.txt",np.round(export,decimals=0), fmt='%i')